# Gemini API: 使用 Python 呼叫函式


<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/google-gemini/gemini-api-cookbook/blob/main/quickstarts/Function_calling.zh.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />在 Google Colab 中執行</a>
  </td>
</table>


函式呼叫讓開發人員可以針對其程式碼中的函式建立描述，然後在要求中傳遞該描述給語言模型。模型產生的回應包含與描述相符的函式名稱和呼叫此函式所需的自變數。函式呼叫讓開發者可以使用函式當作生成式AI應用程式的工具，而且可以在單一要求中定義多個函式。

本筆記本提供程式碼範例以協助你開始使用。


In [ ]:
!pip install -U -q google-generativeai # Install the Python SDK

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.4/137.4 kB 933.2 kB/s eta 0:00:00


In [ ]:
import google.generativeai as genai

## 建立你的 API 金鑰

要執行以下Cell，你的 API 金鑰必須儲存在名為 `GOOGLE_API_KEY` 的 Colab 密碼中。如果你還沒有 API 金鑰，或不確定如何建立 Colab 密碼，請參閱 [驗證](https://github.com/google-gemini/gemini-api-cookbook/blob/main/quickstarts/Authentication.ipynb) Quickstart 作為範例。


In [ ]:
from google.colab import userdata
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

## 函式呼叫基礎


要使用函式呼叫，在建立 [`GenerativeModel`](https://ai.google.dev/api/python/google/generativeai/GenerativeModel) 時，將函式清單傳遞給 `tools` 參數。模型會使用函式名稱、Docstring、參數以及參數類型註解來判定是否需要函式以最佳的方式回答提示。

> 重要事項：SDK 會將函式參數類型註解轉換成 API 所了解的格式 (`glm.FunctionDeclaration`)。API 只支援有限制的參數類型選項，而 Python SDK 的自動轉換只支援以下部分子集：`AllowedTypes = int | float | bool | str | list['AllowedTypes'] | dict`


In [ ]:
def add(a:float, b:float):
    """returns a + b."""
    return a+b

def subtract(a:float, b:float):
    """returns a - b."""
    return a-b

def multiply(a:float, b:float):
    """returns a * b."""
    return a*b

def divide(a:float, b:float):
    """returns a / b."""
    return a*b

model = genai.GenerativeModel(model_name='gemini-1.0-pro',
                              tools=[add, subtract, multiply, divide])

model

genai.GenerativeModel(
    model_name='models/gemini-1.0-pro',
    generation_config={},
    safety_settings={},
    tools=<google.generativeai.types.content_types.FunctionLibrary object at 0x7a99071b61d0>,
)

## 自動函式呼叫


功能呼叫會自然融入 [多輪對話](https://ai.google.dev/api/python/google/generativeai/GenerativeModel#multi-turn) 中，因為它們捕捉使用者和模型之間的來回互動。Python SDK 的 [`ChatSession`](https://ai.google.dev/api/python/google/generativeai/ChatSession) 是聊天絕佳的介面，因為它會替你處理對話記錄，而使用參數 `enable_automatic_function_calling` 會進一步簡化功能呼叫：


In [ ]:
chat = model.start_chat(enable_automatic_function_calling=True)

啟用自動函式呼叫後，如果模型要求，`ChatSession.send_message` 會自動呼叫你的函式。

在下例中，結果看來只是一個包含正確答案的文字回應：


In [ ]:
response = chat.send_message('I have 57 cats, each owns 44 mittens, how many mittens is that in total?')
response.text

'There are 2508 mittens in total.'

In [ ]:
57*44

2508

不過，透過檢視聊天紀錄，你可以看到對話的流程，以及如何將函式呼叫整合到其中。

`ChatSession.history` 屬性會儲存使用者與 Gemini 模型之間對話的時序紀錄。對話中的每個回合都以 [`glm.Content`](https://ai.google.dev/api/python/google/ai/generativelanguage/Content) 物件表示，其中包含下列資訊：

*   **角色** ：識別內容來自「使用者」還是「模型」。
*   **部分** ： [`glm.Part`](https://ai.google.dev/api/python/google/ai/generativelanguage/Part) 物件清單，代表訊息的個別組成部分。透過純文字模型，這些部分可以是：
    *   **文字** ：純文字訊息。
    *   **函式呼叫** ([`glm.FunctionCall`](https://ai.google.dev/api/python/google/ai/generativelanguage/FunctionCall))：模型要求使用提供的引數執行特定函式的要求。
    *   **函式回應** ([`glm.FunctionResponse`](https://ai.google.dev/api/python/google/ai/generativelanguage/FunctionResponse))：使用者在執行要求的函式後所回傳的結果。

在前面有關連指手套計算的範例中，歷史記錄顯示下列順序：

1.  **使用者** ：詢問有關總連指手套數量的問題。
1.  **模型** ：判斷乘法函式是否有幫助，並向使用者送出 FunctionCall 要求。
1.  **使用者** ： `ChatSession` 會自動執行該函式 (因為設定了 `enable_automatic_function_calling`)，並傳回一個包含計算結果的 `FunctionResponse`。
1.  **模型** ：使用函式的輸出結果來構思最後的答案，並將其呈現為文字回應。


In [ ]:
for content in chat.history:
    print(content.role, "->", [type(part).to_dict(part) for part in content.parts])
    print('-'*80)

user -> [{'text': 'I have 57 cats, each owns 44 mittens, how many mittens is that in total?'}]
--------------------------------------------------------------------------------
model -> [{'function_call': {'name': 'multiply', 'args': {'a': 57.0, 'b': 44.0}}}]
--------------------------------------------------------------------------------
user -> [{'function_response': {'name': 'multiply', 'response': {'result': 2508.0}}}]
--------------------------------------------------------------------------------
model -> [{'text': 'There are 2508 mittens in total.'}]
--------------------------------------------------------------------------------


一般的狀態圖如下：

<img src="https://ai.google.dev/images/tutorials/function_call_state_diagram.png" alt="模型總是可以用文字或 FunctionCall 回應。如果模型傳送 FunctionCall，使用者必須用 FunctionResponse 回應" width=50%>

模型可以在回傳文字回應之前回應多個函式呼叫，而函式呼叫會在文字回應之前出現。


## 手動函式呼叫


欲獲得更精準的控制，你可以親自處理模型的 [`glm.FunctionCall`](https://ai.google.dev/api/python/google/ai/generativelanguage/FunctionCall) 請求。這適用於以下情況：

- 在預設 `enable_automatic_function_calling=False` 的 `ChatSession` 中使用。
- 使用 `GenerativeModel.generate_content` (並自己管理聊天記錄)。


以下範例是 Python 中 [Function 呼叫單輪 curl 範例](https://ai.google.dev/docs/function_calling#function-calling-single-turn-curl-sample) 的大致相同版本。它使用回傳 (模擬) 電影播放時間資訊 (可能來自假設的 API) 的功能：


In [ ]:
def find_movies(description: str, location: str = ''):
  """find movie titles currently playing in theaters based on any description, genre, title words, etc.

  Args:
      description: Any kind of description including category or genre, title words, attributes, etc.
      location: The city and state, e.g. San Francisco, CA or a zip code e.g. 95616
  """
  return ['Barbie', 'Oppenheimer']

def find_theaters(location: str, movie: str = ''):
    """Find theaters based on location and optionally movie title which are is currently playing in theaters.

    Args:
        location: The city and state, e.g. San Francisco, CA or a zip code e.g. 95616
        movie: Any movie title
    """
    return ['Googleplex 16', 'Android Theatre']

def get_showtimes(location:str, movie:str, theater:str, date:str):
    """
    Find the start times for movies playing in a specific theater.

    Args:
      location: The city and state, e.g. San Francisco, CA or a zip code e.g. 95616
      movie: Any movie title
      thearer: Name of the theater
      date: Date for requested showtime
    """
    return ['10:00', '11:00']

使用字典以方便稍後按名稱查詢函式。你也可以使用它將函式陣列傳遞至 `GenerativeModel` 的 `tools` 參數。


In [ ]:
functions = {
    'find_movies': find_movies,
    'find_theaters': find_theaters,
    'get_showtimes': get_showtimes,
}

model = genai.GenerativeModel(model_name='gemini-1.0-pro',
                              tools=functions.values())

於使用 `generate_content()` 提出問題後，模型便會要求執行 `function_call`：


In [ ]:
response = model.generate_content('Which theaters in Mountain View show the Barbie movie?')
response.candidates[0].content.parts

[function_call {
  name: "find_theaters"
  args {
    fields {
      key: "location"
      value {
        string_value: "Mountain View, CA"
      }
    }
    fields {
      key: "movie"
      value {
        string_value: "Barbie"
      }
    }
  }
}
]

由於這沒有使用具有自動功能呼叫的 `ChatSession`, 因此必須自己呼叫函式。

執行此操作的一個非常簡單的方法是使用 `if` 語句：

```python
if function_call.name == 'find_theaters':
  find_theaters(**function_call.args)
elif ...
```

不過，由於我們之前建立了 `functions` 字典，因此我們可以將其簡化為：


In [ ]:
def call_function(function_call, functions):
  function_name = function_call.name
  function_args = function_call.args
  return functions[function_name](**function_args)

part = response.candidates[0].content.parts[0]

# Check if it's a function call; in real use you'd need to also handle text
# responses as you won't know what the model will respond with.
if part.function_call:
  result = call_function(part.function_call, functions)

print(result)

['Googleplex 16', 'Android Theatre']


最後，將回應加上訊息歷程傳遞至下一個 `generate_content()` 呼叫，以從模型取得最終文字回應。


In [ ]:
import google.ai.generativelanguage as glm
from google.protobuf.struct_pb2 import Struct

# Put the result in a protobuf Struct
s = Struct()
s.update({'result': result})

# Update this after https://github.com/google/generative-ai-python/issues/243
function_response = glm.Part(
    function_response=glm.FunctionResponse(name='find_theaters', response=s))

# Build the message history
messages = [
    {'role':'user',
     'parts': ['Which theaters in Mountain View show the Barbie movie?.']},
    {'role':'model',
     'parts': response.candidates[0].content.parts},
    {'role':'user',
     'parts': [function_response]}
]

# Generate the next response
response = model.generate_content(messages)
print(response.text)

Theaters showing Barbie in Mountain View, CA are: Googleplex 16, Android Theatre


## 後續步驟


有用的 API 參考：

- [genai.GenerativeModel](https://ai.google.dev/api/python/google/generativeai/GenerativeModel) 類別
  - 其 [GenerativeModel.generate_content](https://ai.google.dev/api/python/google/generativeai/GenerativeModel#generate_content) 方法在幕後建構一個 [glm.GenerateContentRequest](https://ai.google.dev/api/python/google/ai/generativelanguage/GenerateContentRequest)。
    - 要求的 `.tools` 欄位包含一個清單，其中包含 1 [glm.Tool](https://ai.google.dev/api/python/google/ai/generativelanguage/Tool) 物件。
    - 該工具的 `function_declarations` 屬性包含一個 [FunctionDeclarations](https://ai.google.dev/api/python/google/ai/generativelanguage/FunctionDeclaration) 物件清單。
- [response](https://ai.google.dev/api/python/google/ai/generativelanguage/GenerateContentResponse) 可能會包含一個 [glm.FunctionCall](https://ai.google.dev/api/python/google/ai/generativelanguage/FunctionCall)，在 `response.candidates[0].contents.parts[0]` 中。
- 如果設定 `enable_automatic_function_calling`，[genai.ChatSession](https://ai.google.dev/api/python/google/generativeai/ChatSession) 會執行呼叫，並傳回 [glm.FunctionResponse](https://ai.google.dev/api/python/google/ai/generativelanguage/FunctionResponse)。
- 在回應 [FunctionCall](https://ai.google.dev/api/python/google/ai/generativelanguage/FunctionCall) 時，模型始終會預期會收到一個 [FunctionResponse](https://ai.google.dev/api/python/google/ai/generativelanguage/FunctionResponse)。
- 如果你手動使用 [chat.send_message](https://ai.google.dev/api/python/google/generativeai/ChatSession#send_message) 或 [model.generate_content](https://ai.google.dev/api/python/google/generativeai/GenerativeModel#generate_content) 回覆，請記住 API 是無狀態的，你必須傳送整個對話歷程 (一個 [content](https://ai.google.dev/api/python/google/ai/generativelanguage/Content) 物件清單)，而不能只傳送包含 `FunctionResponse` 的最後一個對話。
